In [ ]:
import collections

In [ ]:
%run Conexion_BBDD.ipynb

In [ ]:
df_aerolineas = pd.read_csv("input/aerolineas.csv")
df_aeropuertos = pd.read_csv("input/aeropuertos.csv")
df_countries = pd.read_csv("input/countries.csv")
df_equipamientos = pd.read_csv("input/equipamientos.csv", sep=";")
df_paises = pd.read_excel("input/paises.xls")
df_rutas = pd.read_json("input/rutas.json")

## Carga DM_Pais

In [ ]:
df_paises['cod_continente'] = df_paises['cod_continente'].fillna("NA")

In [ ]:
df_paises = df_paises.drop_duplicates(subset='cod_pais2', keep='first')

In [ ]:
df_paises = pd.merge(left=df_paises, right=df_countries,
                      how='left', left_on='cod_pais2', right_on='country')

In [ ]:
df_pais_def = df_paises[['cod_pais', 
                         'desc_pais', 
                         'cod_pais2',
                         'cod_continente', 
                         'desc_continente', 
                         'longitude', 
                         'latitude', ]]

In [ ]:
df_pais_def = df_pais_def.rename(columns={"desc_pais": "pais", "desc_continente":"continente", "longitude": "longitud",
                                          "latitude": "latitud"})

La siguiente celda puede dar error si se ejecuta por segunda vez

In [ ]:
df_pais_def.to_sql('DM_PAIS', con = conn, if_exists = 'append', index=False)

In [ ]:
%sql select * from DM_PAIS

Comprobaciones de carga

In [ ]:
count_df = df_pais_def.count()
count_df

In [ ]:
count_tab = %sql select count(*) from DM_PAIS
count_tab

In [ ]:
res = %sql select * from DM_PAIS where cod_pais2 in ('AL', 'AD')

In [ ]:
df1 = res.DataFrame()

In [ ]:
df1

In [ ]:
db_query = """
select cod_pais, pais, 
    continente
from DM_PAIS 
    where cod_continente = 'EU';"""

In [ ]:
res = %sql $db_query
res

## Carga DM_AEROPUERTO

In [ ]:
df_aeropuertos[df_aeropuertos["cod_icao"].isnull()].head(10)

In [ ]:
df_aeropuertos["cod_iata_faa"] = df_aeropuertos["cod_iata_faa"].fillna("Z99")

In [ ]:
df_aeropuertos["cod_icao"] = df_aeropuertos["cod_icao"].fillna("Z999")

In [ ]:
aer_pais_list = df_aeropuertos["cod_pais"].unique().tolist()

In [ ]:
pais_list = df_pais_def["cod_pais"].unique().tolist()

Comprobamos que tienen los mismos codigos de pais, obtenemos un resultado negativo

In [ ]:
collections.Counter(aer_pais_list) == collections.Counter(pais_list)

In [ ]:
set_difference = set(aer_pais_list) - set(pais_list)
list_difference = list(set_difference)
list_difference

In [ ]:
df_aeropuertos.loc[df_aeropuertos.cod_pais == 'WAK','cod_pais'] = 'Z99'
df_aeropuertos.loc[df_aeropuertos.cod_pais == 'SSD','cod_pais'] = 'Z99'

In [ ]:
df_aeropuertos_def = df_aeropuertos[['cod_aeropuerto','nombre_aeropuerto','ciudad_aeropuerto','pais_aeropuerto',
                                    'cod_pais','cod_iata_faa','cod_icao','latitud','longitud','altitud','zona_horaria',
                                     'dst','zona_horaria_tz']]

In [ ]:
df_aeropuertos_def.to_sql('DM_AEROPUERTOS', con = conn, if_exists = 'append', index=False)

## Carga DM_AEROLINEAS

In [ ]:
df_aerolineas.head(10)

In [ ]:
df_aerolineas[df_aerolineas["pais_aerolinea"].isnull()].head(10)

In [ ]:
df_aerolineas["cod_iata"] = df_aerolineas["cod_iata"].fillna("Z9")

In [ ]:
df_aerolineas.loc[df_aerolineas.cod_iata == 'Z99','cod_iata'] = 'Z9'

In [ ]:
df_aerolineas.loc[df_aerolineas.sw_activa == 'Y', 'sw_activa'] = 1
df_aerolineas.loc[df_aerolineas.sw_activa == 'N', 'sw_activa'] = 0

In [ ]:
df_aerolineas.to_sql('DM_AEROLINEAS', con = conn, if_exists = 'append', index=False)

## Carga HC_RUTAS

In [ ]:
df_rutas = pd.read_json("input/rutas.json")

In [ ]:
df_rutas.head(2964)

In [ ]:
df_rutas[df_rutas['equipamiento'].isnull()].head(10)

In [ ]:
df_rutas.equipamiento = df_rutas.equipamiento.fillna('Z99')

In [ ]:
df_db_aerolineas = pd.read_sql_query('select cod_aerolinea as cod_aerolinea from DM_AEROLINEAS', conn)

In [ ]:
aero_list = df_db_aerolineas.cod_aerolinea.unique().tolist()

In [ ]:
route_list = df_rutas.cod_aerolinea.unique().tolist()

In [ ]:
set_difference = set(route_list) - set(aero_list)
list_difference = list(set_difference)
list_difference

In [ ]:
df_rutas.loc[df_rutas.cod_aerolinea == '20410','cod_aerolinea'] = 'Z9999'

In [ ]:
df_rutas.head(15)

In [ ]:
df_rutas_def = df_rutas.drop('equipamiento',1)

In [ ]:
df_rutas_def = df_rutas_def[['cod_ruta','cod_aerolinea','cod_aeropuerto_ori','cod_aeropuerto_des']]

In [ ]:
df_rutas_def.to_sql('HC_RUTAS', con = conn, if_exists = 'append', index=False)

## Carga DM_EQUIPAMIENTOS

In [ ]:
df_equipamientos = pd.read_csv("input/equipamientos.csv", sep=";")

In [ ]:
df_equipamientos = df_equipamientos.drop_duplicates(subset='cod_equipamiento', keep='first')

In [ ]:
df_equipamientos

In [ ]:
df_equipamientos.to_sql('DM_EQUIPAMIENTOS', con = conn, if_exists = 'append', index=False)

## Carga CO_RUTAS_EQUIPAMIENTOS

In [ ]:
df_rutas_eq = df_rutas[['cod_ruta','equipamiento']]

In [ ]:
lista_eq = df_rutas['equipamiento'].str.split(" ")
lista_eq

In [ ]:
df_rutas_eq['lista_eq'] = lista_eq

In [ ]:
df_rutas_eq = df_rutas_eq.explode('lista_eq').drop('equipamiento',1)

In [ ]:
df_rutas_eq[df_rutas_eq['cod_ruta'].isin([10903,10904,10905,10906])]

In [ ]:
df_rutas_eq = df_rutas_eq.rename(columns = {'lista_eq':'cod_equipamiento'})

In [ ]:
df_rutas_eq.drop(df_rutas_eq.loc[df_rutas_eq['cod_ruta'] == "Z9999"].index,inplace = True)

In [ ]:
df_rutas_eq.to_sql('CO_RUTAS_EQUIPAMIENTOS', con = conn, if_exists = 'append', index=False)